This was test for wav2vec but started consuming more ram and got too many errors

In [57]:
import os

import torch
from torch._C import device

from torch.utils.data import Dataset
import pandas as pd
import torchaudio
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import json
from sklearn.model_selection import train_test_split

In [58]:
RANDOM_STATE = 42
TEST_RATIO = 0.3
DATA_PATH = 'D:\Programming\Projects\major_project\Dataset\ASR/asr_dataset1.csv'
VOCAB_PATH = "D:\Programming\Projects\major_project\Dataset\ASR/vocab.json"
MODEL_PATH = "D:\Programming\Projects\major_project\Outputs\ASR"

In [59]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [60]:
df = pd.read_csv(DATA_PATH)
df.head()

,Unnamed: 0,filename,filepath,id,labels
0,0,0000c0f43b,Dataset/ASR/openslr/00/0000c0f43b.flac,5bc3b,अवस्थालाई ज्योतिषमा
1,1,00014b6780,Dataset/ASR/openslr/00/00014b6780.flac,c7f92,पूर्वक पूजा आराधना
2,2,0001b0e4b4,Dataset/ASR/openslr/00/0001b0e4b4.flac,693c5,बनेको कुचो
3,3,0001cf394a,Dataset/ASR/openslr/00/0001cf394a.flac,b66ee,प्रकाश पार्न उपमा
4,4,00024e9ee7,Dataset/ASR/openslr/00/00024e9ee7.flac,79c24,दुष्यन्तका नजिक पुग्दछन्


In [61]:
train_dataset, test_dataset = train_test_split(df,test_size=TEST_RATIO, random_state=RANDOM_STATE)

In [62]:
train_dataset.shape

(104991, 5)

In [63]:
test_dataset.shape

(44997, 5)

In [64]:
train_df_1 = train_dataset[0:2000]

train_df_2 = train_dataset[2000:4000]

train_df_3 = train_dataset[4000:6000]

train_df_4 =  train_dataset[6000:8000]

train_df_5 = train_dataset[8000:10000]

test_df = test_dataset[0:2000]

In [73]:
from datasets import Dataset, load_metric

train_data_1 = Dataset.from_pandas(train_df_1)
train_data_2 = Dataset.from_pandas(train_df_2)
train_data_3 = Dataset.from_pandas(train_df_3)
train_data_4 = Dataset.from_pandas(train_df_4)
train_data_5 = Dataset.from_pandas(train_df_5)
test_data = Dataset.from_pandas(test_df)

In [74]:
vocab_dict = open(VOCAB_PATH, "r", encoding="utf8")
output = vocab_dict.read()
print(output)

{"क": 1, "ख": 2, "ग": 3, "घ": 4, "ङ": 5, "च": 6, "छ": 7, "ज": 8, "झ": 9, "ञ": 10, "ट": 11, "ठ": 12, "ड": 13, "ढ": 14, "ण": 15, "त": 16, "थ": 17, "द": 18, "ध": 19, "न": 20, "प": 21, "फ": 22, "ब": 23, "भ": 24, "म": 25, "य": 26, "र": 27, "ल": 28, "व": 29, "श": 30, "ष": 31, "स": 32, "ह": 33, "क्ष": 34, "ज्ञ": 35, "अ": 36, "आ": 37, "इ": 38, "ई": 39, "उ": 40, "ऊ": 41, "ए": 42, "ऐ": 43, "ओ": 44, "औ": 45, "०": 46, "१": 47, "२": 48, "३": 49, "४": 50, "५": 51, "६": 52, "७": 53, "८": 54, "९": 55, "।": 56, "'": 57, "ं": 58, "ँ": 59, "्": 60, "ृ": 61, "ू": 62, "ी": 63, "ौ": 64, "ो": 65, "ा": 66, "ि": 67, "ऋ": 68, "!": 69, "‍": 70, "ै": 71, "ॠ": 72, "े": 73, "‌": 74, "ः": 75, "ु": 76, "|": 0, "[UNK]": 77, "[PAD]": 78}


In [82]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(VOCAB_PATH, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [83]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [84]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [70]:
processor.save_pretrained(MODEL_PATH)

In [2]:
import torch
torch.__version__

'1.12.0+cpu'

In [71]:
import torchaudio 

def convertSpeechFileToArray(batch):
    speech_arr, sampling_rate =  torchaudio.load(batch["filepath"])
    batch["speech"] = speech_arr[0].numpy()
    batch["sampling_rate"] =  sampling_rate
    batch["training_label"] = batch["labels"]
    return batch 



This happened fast

In [75]:
train_data_1 = train_data_1.map(convertSpeechFileToArray, remove_columns=train_data_1.column_names)
train_data_2 = train_data_2.map(convertSpeechFileToArray, remove_columns=train_data_2.column_names)
train_data_3 = train_data_3.map(convertSpeechFileToArray, remove_columns=train_data_3.column_names)
train_data_4 = train_data_4.map(convertSpeechFileToArray, remove_columns=train_data_4.column_names)
train_data_5 = train_data_5.map(convertSpeechFileToArray, remove_columns=train_data_5.column_names)

test_data = test_data.map(convertSpeechFileToArray, remove_columns=test_data.column_names)

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [78]:
import librosa 
import numpy as np 

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]),orig_sr=batch["sampling_rate"],target_sr = 16000)
    batch["sampling_rate"] = 16000
    return batch

This took some time 

In [79]:
train_data_1 = train_data_1.map(resample)
train_data_2 = train_data_2.map(resample)
train_data_3 = train_data_3.map(resample)
train_data_4 = train_data_4.map(resample)
train_data_5 = train_data_5.map(resample)

test_data = test_data.map(resample)

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

# Don't run from Here

In [85]:
def prepare_dataset(batch):
    print(processor)
     # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_label"]).input_ids
    return batch

In [89]:
train_data_1 = train_data_1.map(prepare_dataset, remove_columns=train_data_1.column_names, batch_size=8, num_proc=4, batched=True)
train_data_2 = train_data_2.map(prepare_dataset, remove_columns=train_data_2.column_names, batch_size=8, num_proc=4, batched=True)
train_data_3 = train_data_3.map(prepare_dataset, remove_columns=train_data_3.column_names, batch_size=8, num_proc=4, batched=True)
train_data_4 = train_data_4.map(prepare_dataset, remove_columns=train_data_4.column_names, batch_size=8, num_proc=4, batched=True)
train_data_5 = train_data_5.map(prepare_dataset, remove_columns=train_data_5.column_names, batch_size=8, num_proc=4, batched=True)

test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names, batch_size=8, num_proc=4, batched=True)

NameError: name 'processor' is not defined

In [43]:
df.loc[df['labels']=='अफिसमा भने']

,loc,id,labels
8120,0e00581027,b3c22,अफिसमा भने


In [26]:
df.isna().sum()

loc       0
id        0
labels    0
dtype: int64

In [8]:
all_data_pd = pd.read_csv("openslr_text.csv")
all_data_pd.head()

,loc,id,labels
0,4aa1fdca33,6a6d1,००७ मिलको दूरीमा
1,86e521554a,1f680,००७ मिलको दूरीमा
2,c96b2b1afb,e349f,००७ मिलको दूरीमा
3,2f7ab41953,294d8,०११ देखि काङ्ग्रेसको
4,c290e5124b,88901,०१ सौर द्रव्यमान


In [29]:
all_data_pd.isna().sum()

loc       0
id        0
labels    0
dtype: int64

In [39]:
all_data_pd.loc[all_data_pd['id']=='c0b92']

,loc,id,labels
489,5619da912a,c0b92,१४ अध्याय छन्
786,5c83f493b3,c0b92,१८५४ मा प्रथम
2400,7c3804c749,c0b92,२१ मे १९७९
2592,fc0149a972,c0b92,२६ गतेको हकमा
3196,88887deb68,c0b92,४५ आप्रवासीहरू थिए
...,...,...,...
155292,d8b314d707,c0b92,हामीहरू छोरी ज्वाईँको
155493,b08e4ca9cb,c0b92,हाल ४ पृष्ठमा प्रकाशित
156259,d569e322d7,c0b92,हिँउ हटेपछि श्रावणदेखि
156760,f57671ffda,c0b92,हुने तथा जनसङ्ख्या


In [42]:
all_data_pd.loc[all_data_pd['loc']=='0']

,loc,id,labels
8120,0,b3c22,अफिसमा भने
19886,0,f6b2d,उनले परशुरामलाई यस्तो
23498,0,22333,एउटा बयान
36351,0,09eb9,क्रुद्ध भइन्
37960,0,90c1c,खुर भएका जनावरहरूलाई
81094,0,4d069,पनिर बिक्री हुन्छ
83853,0,963b9,पहिलो लक्ष्य हुने
84072,0,056c7,पाउन सक्छौं
84263,0,8272f,पाखो बिराउन थाले
95612,0,9f669,बसेर खान्छन्


In [27]:
all_data_pd.sort_values(by='loc')

,loc,id,labels
127810,0,c0b92,रूपान्तर गर्नु
140670,0,fd59f,सञ्चालन गर्दछ्न्
81094,0,4d069,पनिर बिक्री हुन्छ
84072,0,056c7,पाउन सक्छौं
95612,0,9f669,बसेर खान्छन्
...,...,...,...
37788,fffe93ed54,09eb9,खाल्डो खनी पोखरीको
130255,ffff8b1260,e4596,लामो हुन्छ
126935,ffffc0c531,d0cd7,रिहा गराउनको आमाग
114317,fffff63362,7ef78,यस पर्वमा चोखो


In [12]:
train_pd = pd.read_csv("shuffled_csv.csv")
train_pd.head()

,Unnamed: 0,Unnamed: 0.1,loc,id,labels,loc_valid
0,0,72325,../input/large-nepali-asr/openslr/6f/6f0ac1d1c...,5cef8,नगर छाडेर बनौधिया,True
1,2,31102,../input/large-nepali-asr/openslr/4b/4ba42b198...,ce4af,कार्यालयले गरेको छ,True
2,4,107894,../input/large-nepali-asr/openslr/59/59667de30...,1d75d,मानिसले गोमन सर्पलाई,True
3,5,134193,../input/large-nepali-asr/openslr/60/60d456868...,2a5b1,विज्ञानी जेम्स क्लर्क,True
4,6,35189,../input/large-nepali-asr/openslr/fb/fbce02b3a...,33783,केही ब्रतालुहरू रातभर,True


In [13]:
train_pd.sort_values(by='loc')

,Unnamed: 0,Unnamed: 0.1,loc,id,labels,loc_valid
65939,69349,10174,../input/large-nepali-asr/openslr/00/0000c0f43...,5bc3b,अवस्थालाई ज्योतिषमा,True
130624,137472,86943,../input/large-nepali-asr/openslr/00/00014b678...,c7f92,पूर्वक पूजा आराधना,True
88633,93262,94921,../input/large-nepali-asr/openslr/00/0001b0e4b...,693c5,बनेको कुचो,True
113707,119655,88603,../input/large-nepali-asr/openslr/00/0001cf394...,b66ee,प्रकाश पार्न उपमा,True
52879,55587,69055,../input/large-nepali-asr/openslr/00/00024e9ee...,79c24,दुष्यन्तका नजिक पुग्दछन्,True
...,...,...,...,...,...,...
11515,12120,37788,../input/large-nepali-asr/openslr/ff/fffe93ed5...,09eb9,खाल्डो खनी पोखरीको,True
6915,7278,130255,../input/large-nepali-asr/openslr/ff/ffff8b126...,e4596,लामो हुन्छ,True
73247,77034,126935,../input/large-nepali-asr/openslr/ff/ffffc0c53...,d0cd7,रिहा गराउनको आमाग,True
78793,82866,114317,../input/large-nepali-asr/openslr/ff/fffff6336...,7ef78,यस पर्वमा चोखो,True


In [44]:
train_pd.isna().sum()

AttributeError: ignored

In [ ]:
y =  train_pd["labels"]
x = train_pd.drop(["Unnamed: 0","Unnamed: 0.1","labels"],axis=1)
print(x.columns)

Index(['loc', 'id', 'loc_valid'], dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(train_pd,test_size=0.3, random_state=42)

In [ ]:
vocab_dict = open("vocab.json", "r")
output = vocab_dict.read()
print(output)

{"क": 1, "ख": 2, "ग": 3, "घ": 4, "ङ": 5, "च": 6, "छ": 7, "ज": 8, "झ": 9, "ञ": 10, "ट": 11, "ठ": 12, "ड": 13, "ढ": 14, "ण": 15, "त": 16, "थ": 17, "द": 18, "ध": 19, "न": 20, "प": 21, "फ": 22, "ब": 23, "भ": 24, "म": 25, "य": 26, "र": 27, "ल": 28, "व": 29, "श": 30, "ष": 31, "स": 32, "ह": 33, "क्ष": 34, "ज्ञ": 35, "अ": 36, "आ": 37, "इ": 38, "ई": 39, "उ": 40, "ऊ": 41, "ए": 42, "ऐ": 43, "ओ": 44, "औ": 45, "०": 46, "१": 47, "२": 48, "३": 49, "४": 50, "५": 51, "६": 52, "७": 53, "८": 54, "९": 55, "।": 56, "'": 57, "ं": 58, "ँ": 59, "्": 60, "ृ": 61, "ू": 62, "ी": 63, "ौ": 64, "ो": 65, "ा": 66, "ि": 67, "ऋ": 68, "!": 69, "‍": 70, "ै": 71, "ॠ": 72, "े": 73, "‌": 74, "ः": 75, "ु": 76, "|": 0, "[UNK]": 77, "[PAD]": 78}


Todo: Read about tokenization

unk_token => Unknown Token inorder to be converted to an ID
pad_token => Padding Token


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")